Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с всех сайтов. Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

In [457]:
#Сделаем функцию парсинга отдельно
def PageParser_HH (url, params , headers):
  vacancies_list = []
  #Получение DOM с сайта
  response = requests.get(url=url, params=params, headers=headers)
  #Извлечение искомых полей
  soup = bs(response.text, 'html.parser')
  vacancies = soup.find_all('div', {'class': 'serp-item'})
  #Оформление списка словарей вакансий  
  for vacancy in vacancies:
    vacancy_name = vacancy.find('a', {'class': 'serp-item__title'}).getText()
    vacancy_link = vacancy.find('a', {'class': 'serp-item__title'})['href']
    vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if vacancy_salary is not None:
      vacancy_salary = vacancy_salary.getText()

    vacancy_dict = {
        'name': vacancy_name,
        'link': vacancy_link,
        'salary': vacancy_salary,
        'source': 'HeadHunter (https://hh.ru/)',
        }
    vacancies_list.append(vacancy_dict)
  return vacancies_list

In [458]:
# Функция преобразовывающая строку зарплаты в словарь
def Salary_trace(salary):
  if salary != None:
  #Возьмем строку и очистим от лишних символов
   string = salary.replace('\u202f','').replace(' ','')
   #Вычленим валюту
   currency = string
   for c in ['0' , '1' ,'2' ,'3' ,'4' ,'5' ,'6' ,'7' ,'8' ,'9' ,'от' , 'до' , '–']:
      currency = currency.replace(c,'')

  #Вычленим минимальную ЗП
   if string.find('от') > -1:
     From = int(''.join(i for i in string if not ( i.isalpha() or i =='.')))
   elif string.find('–') > -1:
     From = int((string[:string.find("–")]))
   else: 
     From = None

  #Вычленим максимальную ЗП
   if string.find('до') > -1:
     To = int(''.join(i for i in string if not ( i.isalpha() or i =='.')))
   elif string.find('–') > -1:
     To = int((string[string.find("–")+1:string.find(currency)]))
   else: 
     To = None

  else:
   From = None
   To = None
   currency = None

  salary_dict = {'From': From , 'To' : To , 'currency' : currency }

  return salary_dict


In [459]:
from bs4 import BeautifulSoup as bs
import requests

In [460]:
url = 'https://hh.ru/search/vacancy?'
NotEmpty = True
SearchPage = 6
vacancies_list = []

params = {
    'area': 1,
    'text': 'python',
    'items_on_page': 20,
    'experience' : 'noExperience',
    'page' : SearchPage
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.124',
}

In [461]:
# Перебор страниц сайта с наполнением списка вакансий
while NotEmpty:
  vacancies_list_page = []
  vacancies_list_page = PageParser_HH(url, params, headers)
  vacancies_list = vacancies_list + vacancies_list_page
  params["page"] +=1  #Переход на следующую страницу поиска
  if len(vacancies_list_page) == 0: # Проверка на наличие результата
    NotEmpty = False


In [462]:
# Преобразуем зарплаты в словари зарплат
for i in range(len(vacancies_list)):
  vacancies_list[i]['salary'] = Salary_trace(vacancies_list[i]['salary'])
  # из словаря извлечем в ключи
  vacancies_list[i]['salary_From'] = vacancies_list[i]['salary']['From']
  vacancies_list[i]['salary_To'] = vacancies_list[i]['salary']['To']
  vacancies_list[i]['salary_currency'] = vacancies_list[i]['salary']['currency']
  del vacancies_list[i]['salary']


Подготовим из результата датафрейм и запишем его в файл json

In [463]:
import pandas as pd

In [464]:
df = pd.DataFrame( vacancies_list ) 
print(df)

                                                 name  \
0   Ночной специалист службы поддержки с техническ...   
1                                  Junior C developer   
2   Эксперт отдела надзора (Департамент микрофинан...   
3                             Quantitative Researcher   
4                               Стажер-разработчик C#   
5            Преподаватель программирования для детей   
6             Team Lead команды репутационная система   
7   Старший разработчик Go мониторинг, Платформа, ...   
8   Аналитик-разработчик в команду технологий безо...   
9                                  Стажер Sber AI Lab   
10                   Junior Product Analyst (Gamedev)   
11                   Специалист технической поддержки   
12                   Менеджер отдела ценовой политики   
13       Младший аналитик (управленческая отчетность)   
14  Преподаватель компьютерных/IT курсов, программ...   
15                   Инженер по защите web приложений   
16                             

In [465]:
#Сохраним файл json
df.to_json('vacancies.json')